In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
COLUMNA_GRD_INTERNO = [
    "rut",
    "RUT_LIMPIO",
    "sexo_cod",
    "prevision_cod",
    "hospital_de_procedencia_cod",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_cod",
    "diag_01_principal_des",
    "diag_02_secundario_cod",
    "diag_02_secundario_des",
    "diag_03_secundario_cod",
    "diag_03_secundario_des",
    "proced_01_principal_cod",
    "proced_01_principal_des",
    "1ra_intervencion_cirujano",
    "especialidad_medica_de_la_intervencion_des",
]

RUTA_GRD_INTERNO = "../data/processed/grd_interno.csv"
COLUMNAS_FECHA_GRD_INT = ["fecha_ingreso", "fecha_de_egreso"]

In [ ]:
# Carga la base de datos de GRD Interno
df_grd_interno = build_features.cargar_datos_grd(RUTA_GRD_INTERNO, columnas=COLUMNA_GRD_INTERNO)
df_grd_interno = build_features.formatear_columnas_fecha(df_grd_interno, COLUMNAS_FECHA_GRD_INT)
df_grd_interno = df_grd_interno.reset_index(names="id_egreso")
df_grd_interno["RUT_LIMPIO"] = df_grd_interno["RUT_LIMPIO"].astype(str)
df_grd_interno["diag_01_principal_cod"] = (
    df_grd_interno["diag_01_principal_cod"].str.replace(".", "", regex=False).str.ljust(4, "X")
)

# Carga la base de datos de Procedimientos - Datos Estadisticos
procedimientos = pd.read_csv("../data/external/procedimientos_procesada.csv")
procedimientos["id_paciente"] = procedimientos["id_paciente"].astype(str)

In [ ]:
# Obtiene el ultimo diagnostico de un paciente por GRD
diagnosticos_pacientes = (
    df_grd_interno.groupby("RUT_LIMPIO")["diag_01_principal_cod"].unique().str[-1]
)

In [ ]:
procedimientos.merge(diagnosticos_pacientes, how="left", left_on="id_paciente", right_index=True)